In [ ]:
from bach_open_taxonomy import ObjectivFrame

In [ ]:
time_aggregation = 'YYYY-MM-DD'

In [ ]:
df = ObjectivFrame.from_objectiv_data(time_aggregation=time_aggregation, start_date='2021-11-16')

In [ ]:
df.head()

# model hub structure:
* `map` functions: adds a column, same number of rows as df. They can be added to the df as they have:
  * same base node. *or*,
  * same index
    * future: always same index, so you can also add it to df after df is materialized
* `filter`: function, boolean series with same index can be used to filter the df; results in less rows
* `aggregate` functions: returns aggregated data; different index than original df. also takes a `filter` argument.
  * future: functions from `aggregate` can also be used as `map` function; so it adds a column to df with results of the `aggregate` model.
  
  

Just 19 cells

## map

In [ ]:
df.model_hub.map.is_first_session().head()

In [ ]:
df['ifs'] = df.model_hub.map.is_first_session()
df.head()

In [ ]:
new_users = df.model_hub.map.is_new_user()
new_users.head()

In [ ]:
df.add_conversion_event(location_stack=df.location_stack.json[{'id': 'page-about', '_type': 'SectionContext'}:],
                        event_type='ClickEvent',
                        name='about_clicks')

In [ ]:
df.mh.map.conversion_count('about_clicks').head()

## aggregate

In [ ]:
df.model_hub.aggregate.unique_users().head()

In [ ]:
df.model_hub.aggregate.unique_users(filter=new_users).head()

In [ ]:
# divide with different base nodes
ratio = df.model_hub.aggregate.unique_users(filter=new_users) / df.model_hub.aggregate.unique_users()
ratio.head()

## filter

In [ ]:
df.moment.count().head()

In [ ]:
new_df = df.mh.filter(df.model_hub.map.is_new_user())

In [ ]:
new_df.moment.count().head()

In [ ]:
df.model_hub.filter(new_users).model_hub.aggregate.unique_users().head()

## stack

In [ ]:
df.mh.agg.unique_users(filter=df.mh.map.is_conversion_event('about_clicks')).to_pandas()

In [ ]:
df.mh.agg.unique_users(filter=df.mh.map.conversion_count(name='about_clicks')==1).to_pandas()

In [ ]:
df.mh.agg.unique_users(filter=df.mh.map.conversion_count(name='about_clicks')==2).to_pandas()

In [ ]:
twice_converted = df.mh.agg.unique_users(filter=df.mh.map.conversion_count(name='about_clicks')==2) / df.mh.agg.unique_users()
twice_converted.head()

In [ ]:
# stackin': unique new users that converted three times in a session
df.mh.filter(df.mh.map.is_new_user(time_aggregation = 'YYYY-MM-DD'))\
  .mh.filter(df.mh.map.conversion_count(name='about_clicks')==3)\
  .mh.agg.unique_users()\
  .head()

In [ ]:
# stackin' cont'd: filter last two hits before converting
df.mh.map.pre_conversion_hit_number(name='about_clicks').head()

In [ ]:
df.mh.filter(df.mh.map.pre_conversion_hit_number(name='about_clicks')<5).head()